`Alexey Tkachenko BS22 AAI-02`

## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

---

In [ ]:
!pip install pandas nltk autocorrect

## Loading the data

I decided to use the provided datasets for bigrams. The I picked were the `bigrams.txt` and `coca_all_links`. I didn't quite get the extra info that is present in the `coca_all_links`, but decided to use it anyway. Just without the unknown columns.

No funny business here, just reading the data

In [1]:
import os
import re
import pandas as pd

def load_datasets(datadir: str):
    df = pd.DataFrame(columns=["count", "w1", "w2"])
    for path in os.listdir(datadir):
        filepath = os.path.join(datadir, path)

        new_df = pd.read_csv(
            filepath,
            sep="\t",
            usecols=[0, 1, 2],
            names=["count", "w1", "w2"],
            encoding="latin-1",
        )

        new_df["w1"] = new_df["w1"].str.lower()
        new_df["w2"] = new_df["w2"].str.lower()

        df = pd.concat([df, new_df], ignore_index=True)

    return df, df['count'].sum()

In [2]:
df, total_count = load_datasets('data')
df

,count,w1,w2
0,36,a-national,rank
1,92,abandoned,building
2,139,abandoned,buildings
3,50,abandoned,car
4,47,abandoned,cars
...,...,...,...
1176689,24,zviad,gamsakhurdia
1176690,25,zweimal,leben
1176691,24,zwick,and
1176692,24,zydeco,music


I also return the `total_count` that I will use for probabilities

## Candidate selection algorithm

Initially I used Norvig's approach for candidate selection. It was sufficient, but during testing I found that the even though the candidate selection algorithm is good, the quality of the dataset for known words matters most. I used `english_words` package to get a dictionary of real english words, but it was of low quality.

While looking for a decent dataset, I found an amazing library called [autocorrect](https://github.com/filyp/autocorrect). I decided to implement the correction myself, but take their candidate selector. If you take a look at the source code, they take an approach similar to Norvig's:

```py
def get_candidates(self, word):
    w = Word(word, self.lang, self.only_replacements)
    if self.fast:
        candidates = self.existing([word]) or self.existing(w.typos()) or [word]
    else:
        candidates = (
            self.existing([word])
            or self.existing(w.typos())
            or self.existing(w.double_typos())
            or [word]
        )
    return [(self.nlp_data.get(c, 0), c) for c in candidates]
```

I decided to go forward with this approach, since it showed much better error detection rates.

In [3]:
import autocorrect
spell = autocorrect.Speller()
def candidates(word: str):
    return sorted(spell.get_candidates(word), key=lambda e: e[0], reverse=True)

Here I have a function that utilized the bigram dataset we've loaded earlier. I basically query the dataframe for bigrams based on two words, and return the probability. I normalize probabilities using `np.log10`, which I read about [here](https://www.kaggle.com/code/dhruvdeshmukh/spelling-corrector-using-n-gram-language-model).

I build an index to save on computations. Surprizingly, this works much faster than I expected. The lookup speed on `pd.DataFrame()` is outstanding!.

In [4]:
import numpy as np


def build_index(df: pd.DataFrame):
    df["comb"] = df["w1"] + df["w2"]

    return (
        df.drop(["w1", "w2"], axis=1).set_index(["comb"]).drop_duplicates(keep="first")
    )


index = build_index(df)


def prob(w1, w2):
    try:
        return np.log10(index[w1 + w2][0] / total_count)
    except KeyError:
        return -np.inf

The idea behind the algorithm is as follows:

- We pad the sentence with `[BOS]` and `[EOS]` tokens for convenience
- We generate bigrams 
- Iterating through bigrams, take the candidate with the highest score
- Take top 3 candidates
- Calculate the probability of new bigrams with one of the new candidates replacing the first word
- Take most probable one

Also:
- Compute the same thing, but for the second word in the bigram (replacing the second word instead of the first)
- Save the this candidate for the next step of the loop
- During the next iteration, if the candidate from the previous step is more probable, make the change

This addition helps the algorithm "look" at the word from "both sides" and helped in some cases during testing. The bigram might have looked ok from the left, but from the right it was incorrect. That is why I have implemented this change.

In [6]:
from nltk import ngrams

def correct(sentence: str, n=2, b=3):
    w2_incorrect_best_for_w1 = (-np.inf, "")

    # words = ["[BOS]"] + sentence.lower().split() + ["[EOS]"]
    words =  ["[BOS]"] + sentence.strip().lower().split() + [
        "[EOS]"
    ]  # Assuming the first and last words are correct (for now)
    ngrms = list(ngrams(words, n))

    correct_sequience = []
    for gram in ngrms:
        w1, w2 = gram


        cands = []
        for cand in sorted(candidates(w1), key=lambda e: e[0], reverse=True)[
            :b
        ]:  # Top 3 candidates
            _, c = cand
            cands.append((prob(c, w2), c))
        cands = sorted(cands, key=lambda e: e[0], reverse=True)

        # print(f"{w1} {w2} | {cands}")

        correct_sequience += [cands[0][1] if cands[0][0] > w2_incorrect_best_for_w1[0] else w2_incorrect_best_for_w1[1]]
            

        cands = []
        for cand in sorted(candidates(w2), key=lambda e: e[0], reverse=True)[
            :b
        ]:  # Top 3 candidates
            _, c = cand
            cands.append((prob(w1, c), c))
        cands = sorted(cands, key=lambda e: e[0], reverse=True)

        # print(f"{w1} {w2} | {cands}")
        
        w2_incorrect_best_for_w1 = cands[0]

        # correct_sequience += [cands[0][1]]
            
            

    return " ".join(correct_sequience).strip()


print(correct("we wll be dking sport and yow need to leave their dking man"))
print(correct("ok i understood your poimr but it is stoll hard for me to believe rhat he made it"))
print(correct("does not duvude the sinsay from the week"))
print(correct("Goof now sit drwn and tell me, he that lnows"))

we all be doing sport and you need to leave their doing man
ok i understood your point but it is still hard for me to believe that he made it
does not divide the sunday from the week
good now sit down and tell me he that knows


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [7]:
from collections import Counter


def words(text):
    return re.findall(r"\w+", text.lower())


WORDS = Counter(words(open("big.txt").read()))


def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N


def correction(word):
    "Most probable spelling correction for word."
    return max(candidates_n(word), key=P)


def candidates_n(word):
    "Generate possible spelling corrections for word."
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]


def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


def edits1(word):
    "All edits that are one edit away from `word`."
    letters = "abcdefghijklmnopqrstuvwxyz"
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [8]:
import random
import re


def preprocess(text: list[str]):
    new_text = []
    for line in text:
        new_text.append(re.sub(r"[^A-Za-z0-9 ]+", "", line.lower()).strip())

    return new_text


def noizyfy(text: list[str], error_rate=0.25, error_type_rate=0.25):
    new_text = []
    for line in text:
        words = line.split()

        new_words = []
        for w in words:

            def mutate(word):
                def edits1(word):
                    "All edits that are one edit away from `word`."
                    letters = "abcdefghijklmnopqrstuvwxyz"
                    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
                    deletes = [L + R[1:] for L, R in splits if R]
                    transposes = [
                        L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1
                    ]
                    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
                    inserts = [L + c + R for L, R in splits for c in letters]
                    return set(deletes + transposes + replaces + inserts)

                def edits2(word):
                    "All edits that are two edits away from `word`."
                    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

                if random.random() < error_type_rate:
                    return random.choice(list(edits1(word)))
                else:
                    return random.choice(list(edits2(word)))

            if random.random() < error_rate:
                new_words.append(mutate(w))
            else:
                new_words.append(w)

        new_text.append(" ".join(new_words))
    return new_text


# Original text: https://shakespeare.mit.edu/hamlet/full.html
shakespeare_text = [
    "Disasters in the sun; and the moist star",
    "Upon whose influence Neptune's empire stands",
    "Was sick almost to doomsday with eclipse:",
    "And even the like precurse of fierce events,",
    "As harbingers preceding still the fates",
    "And prologue to the omen coming on,",
    "Have heaven and earth together demonstrated",
    "Unto our climatures and countrymen.--",
    "But soft, behold! lo, where it comes again!",
]


text = preprocess(shakespeare_text)


for original, fake in zip(text, noizyfy(text)):
    print("Errored out:")
    print(fake)
    print("N-grams")
    print(correct(fake))
    print("Norvig")
    print(" ".join(correction(word) for word in fake.split()))
    print("Original:")
    print(original)
    print()

Errored out:
djsasters ins the sun and gtlhe moist star
N-grams
disasters ins the sun and the moist star
Norvig
disasters in the sun and the moist star
Original:
disasters in the sun and the moist star

Errored out:
upon whaose influence neptunes empire stands
N-grams
upon whose influence neptunes empire stands
Norvig
upon whose influence neptunes empire stands
Original:
upon whose influence neptunes empire stands

Errored out:
uals sick almost twq geomsday wrth eclipse
N-grams
urls sick almost two doomsday with eclipse
Norvig
pals sick almost two geomsday with eclipses
Original:
was sick almost to doomsday with eclipse

Errored out:
and even the like precurse of fierce events
N-grams
and even the like precise of fierce events
Norvig
and even the like recourse of fierce events
Original:
and even the like precurse of fierce events

Errored out:
as hqrbingeos precqeding nstinl the fates
N-grams
as hqrbingeos preceding still the fates
Norvig
as harbingers preceding still the fates
Origina

On synthetic errors both models perform very poorly. Let's try something more realistic.

In [9]:
adj_keys = {
    "q": ["w", "a", "s"],
    "w": ["q", "e", "a", "s", "d"],
    "e": ["w", "r", "s", "d", "f"],
    "r": ["e", "t", "d", "f", "g"],
    "t": ["r", "y", "f", "g", "h"],
    "y": ["t", "u", "g", "h", "j"],
    "u": ["y", "i", "h", "j", "k"],
    "i": ["u", "o", "j", "k", "l"],
    "o": ["i", "p", "k", "l"],
    "p": ["o", "l"],
    "a": ["q", "w", "s", "z", "x"],
    "s": ["a", "w", "e", "d", "z", "x", "c"],
    "d": ["s", "e", "r", "f", "x", "c", "v"],
    "f": ["d", "r", "t", "g", "c", "v", "b"],
    "g": ["f", "t", "y", "h", "v", "b", "n"],
    "h": ["g", "y", "u", "j", "b", "n", "m"],
    "j": ["h", "u", "i", "k", "n", "m"],
    "k": ["j", "i", "o", "l", "m"],
    "l": ["k", "o", "p"],
    "z": ["a", "s", "x"],
    "x": ["z", "s", "d", "c"],
    "c": ["x", "d", "f", "v"],
    "v": ["c", "f", "g", "b"],
    "b": ["v", "g", "h", "n"],
    "n": ["b", "h", "j", "m"],
    "m": ["n", "j", "k"],
}


def noizyfy(text: list[str], error_rate=0.10):
    new_text = []
    for line in text:
        words = line.split()

        new_words = []
        for w in words:

            def mutate(word):
                def edits1(word):
                    "All edits that are one edit away from `word`."
                    letters = "abcdefghijklmnopqrstuvwxyz"
                    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
                    deletes = [L + R[1:] for L, R in splits if R]
                    transposes = [
                        L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1
                    ]
                    replaces = [
                        L + random.choice(adj_keys[R[0]]) + R[1:]
                        for L, R in splits
                        if R
                    ]
                    return set(deletes + transposes + replaces)

                # def edits2(word):
                #     "All edits that are two edits away from `word`."
                #     return (e2 for e1 in edits1(word) for e2 in edits1(e1))

                # if random.random() < error_type_rate:
                return random.choice(list(edits1(word)))
                # else:
                #     return random.choice(list(edits2(word)))

            if random.random() < error_rate:
                new_words.append(mutate(w))
            else:
                new_words.append(w)

        new_text.append(" ".join(new_words))
    return new_text


text = [
    "The cat leaped gracefully onto the windowsill, its tail flicking in the sunlight.",
    "She decided to bake a cake, even though she had never tried the recipe before.",
    "The old bookstore smelled of dust and memories, with shelves stacked to the ceiling.",
    "He couldn’t believe his luck when he found a twenty-dollar bill on the sidewalk.",
    "The storm rolled in quickly, turning the sky a deep shade of gray.",
]


text = preprocess(text)


for original, fake in zip(text, noizyfy(text)):
    print("Errored out:")
    print(fake)
    print("N-grams")
    print(correct(fake))
    print("Norvig")
    print(" ".join(correction(word) for word in fake.split()))
    print("Original:")
    print(original)
    print()

Errored out:
the act leaped gracefully onto he windowsill its tail flicking in the sunlight
N-grams


the act leased peacefully onto he windowsill its tail clicking in the sunlight
Norvig
the act leaped gracefully onto he windowsill its tail flicking in the sunlight
Original:
the cat leaped gracefully onto the windowsill its tail flicking in the sunlight

Errored out:
shw decided to bake a cake even though she had never triex the rceipe before
N-grams
sha decided to bake a cake even though she had never tried the recipe before
Norvig
she decided to bake a cake even though she had never tried the recipe before
Original:
she decided to bake a cake even though she had never tried the recipe before

Errored out:
the old bookstore smelled of duts and memories with shelves stacked to the ceiling
N-grams
the old bookstore spelled of duty and memories with shelves stacked to the ceiling
Norvig
the old bookstore smelled of duty and memories with shelves stacked to the ceiling
Original:
the old bookstore smelled of dust and memories with shelves stacked to the ceiling

Errored out:
he couldnt be

# Conclusions
The ngram language model seem to do a better job if we consider cases closer to real-life with typing errors, though the Norvig's approach is not so far behind. To me, it seems like that for the n-gram model the quality of the n-gram dataset and the dictionary of known words matter significantly. Regardless, I believe that we should look towards more advanced solutions like transformers, since the capture the meaning of tokens, not just their frequency in text.